# Working with Time Zones

## Creating and working with time-zone objects

The standard way to create a datetime literal is to attach it to the constructor by passing it to the `tzinfo` argument of the constructor:

In [1]:
from datetime import datetime, timezone, timedelta
from dateutil import tz

In [2]:
from helper_functions import print_dt_tzinfo

In [3]:
EASTERN = tz.gettz("America/New_York")

In [4]:
dt = datetime(2017, 8, 11, 14, tzinfo=EASTERN)
print_dt_tzinfo(dt)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


If you have a naïve wall time or a wall time in another zone that you want to translate without shifting, use `datetime.replace`:

In [5]:
print_dt_tzinfo(dt.replace(tzinfo=tz.gettz('America/Los_Angeles')))

2017-08-11 14:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


If you have an *absolute* time, in UTC or otherwise, and you want to represent it in another timezone, use `datetime.astimezone`:

In [6]:
print_dt_tzinfo(dt.astimezone(tz.gettz('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


### `pytz`

In `pytz`, `datetime.astimezone()` still works as expected:

In [7]:
import pytz

In [8]:
print_dt_tzinfo(dt.astimezone(pytz.timezone('America/Los_Angeles')))

2017-08-11 11:00:00-0700
    tzname:   PDT;      UTC Offset:  -7.00h;        DST:      1.0h


But the constructor and `.replace` methods fail horribly:

In [9]:
print_dt_tzinfo(dt.replace(tzinfo=pytz.timezone('America/Los_Angeles')))

2017-08-11 14:00:00-0753
    tzname:   LMT;      UTC Offset:  -7.88h;        DST:      0.0h


This is because `pytz` uses a different time zone model. `pytz`'s time zone model implements the `tzinfo` interface *statically*, which is to say that the `tzname`, `utcoffset` and `dst` are all calculated eagerly, when the `tzinfo` is attached to the `datetime`.

In order to accomplish this, `pytz` expects all `tzinfo` objects to be attached to the `datetime` *by the time zone object itself*, using the `localize()` method:

In [10]:
EASTERN_p = pytz.timezone('America/New_York')
dt_p = EASTERN_p.localize(datetime(2017, 8, 11, 14))
print_dt_tzinfo(dt_p)

2017-08-11 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


This also means that unlike with normal `tzinfo` objects, after you've done some arithmetic on a `pytz`-aware `datetime` object, you must `normalize` it:

In [11]:
print('dateutil:')
print_dt_tzinfo(dt + timedelta(days=180))
print('')
print('pytz')
print_dt_tzinfo(dt_p + timedelta(days=180))

dateutil:
2018-02-07 14:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

pytz
2018-02-07 14:00:00-0400
    tzname:   EDT;      UTC Offset:  -4.00h;        DST:      1.0h


In [12]:
print_dt_tzinfo(EASTERN_p.normalize(dt_p + timedelta(days=180)))

2018-02-07 13:00:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


### `datetime.now` and `datetime.fromtimestamp`

The alternate constructors `now()` and `fromtimestamp` both take an optional argument `tz`:


```python
    def now(tz=None):
        """Get the datetime representing the current time"""
        # ...

    def fromtimestamp(self, timestamp, tz=None):
        """ Return the datetime corresponding to a POSIX timestamp """
        # ...
```

If the `tz` parameter is not passed, they will return a *naïve* datetime, representing the time in your system local time.

In [13]:
datetime.fromtimestamp(1577854800.0)

datetime.datetime(2019, 12, 31, 22, 0)

If you want an *aware* timezone from a timestamp (or the current time), pass it to the `tz` parameter, and it will calculate the correct `datetime` matching that time zone. This works with both `pytz` and `dateutil.tz`:

In [14]:
print("pytz:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN_p))
print("")
print("dateutil:")
print_dt_tzinfo(datetime.fromtimestamp(1577845800.0, tz=EASTERN))

pytz:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h

dateutil:
2019-12-31 21:30:00-0500
    tzname:   EST;      UTC Offset:  -5.00h;        DST:      0.0h


**Note**: There are also the semi-deprecated `datetime.utcnow()` and `datetime.utcfromtimestamp()` functions. These  return a *naïve* datetime expressed in UTC. It is almost always better to simply pass `tz=UTC` (where `UTC` is some time zone object), which will give you an *aware* `datetime`.

### Exercise: Current time in multiple time zones


To practice using time zones, try writing a function that takes as inputs a list of IANA time zones and prints the time in each time zone.

**NOTE**: Multiple calls to `datetime.now` will give you *different* answers for each time zone. Try to accomplish this with only *one* call to `datetime.now`.

So, for example:

```python
>>> now_in_zones(['Asia/Tokyo',
...               'Europe/Berlin',
...               'America/New_York',
...               'America/Los_Angeles'])
```
```
Asia/Tokyo:               2020-01-01 14:00:00+09:00
Europe/Berlin:            2020-01-01 06:00:00+01:00
America/New_York:         2020-01-01 00:00:00-05:00
America/Los_Angeles:      2019-12-31 21:00:00-08:00
```

In [31]:
from dateutil import tz

In [19]:
Tokyo = tz.gettz('Asia/Tokyo')
dir(Tokyo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_filename',
 '_find_last_transition',
 '_find_ttinfo',
 '_fold',
 '_fold_status',
 '_fromutc',
 '_get_ttinfo',
 '_read_tzfile',
 '_resolve_ambiguous_time',
 '_set_tzdata',
 '_trans_idx',
 '_trans_list',
 '_trans_list_utc',
 '_ttinfo_before',
 '_ttinfo_dst',
 '_ttinfo_first',
 '_ttinfo_list',
 '_ttinfo_std',
 'dst',
 'fromutc',
 'is_ambiguous',
 'tzname',
 'utcoffset']

In [33]:
def now_in_zones(tz_list):
    now = datetime.now(tz=tz.UTC)
    for zone in tz_list:
        t = tz.gettz(zone)
        dt = now.astimezone(t)
        print(f"{zone + ':':<25} {dt}")

now_in_zones(['Asia/Tokyo',
              'Europe/Berlin',
              'America/New_York',
              'America/Los_Angeles'])
    

Asia/Tokyo:               2019-05-02 02:58:39.961165+09:00
Europe/Berlin:            2019-05-01 19:58:39.961165+02:00
America/New_York:         2019-05-01 13:58:39.961165-04:00
America/Los_Angeles:      2019-05-01 10:58:39.961165-07:00
